In [1]:

import pandas as pd
import numpy as np

In [2]:
## cleaning tool


def perc_null(X):
    
    total = X.isnull().sum().sort_values(ascending=False)
    data_types = X.dtypes
    percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)

    missing_data = pd.concat([total, data_types, percent], axis=1, keys=['Total','Type' ,'Percent'])
    return missing_data


In [3]:
##note KNN or other clusters might be helpful group the teams in smart way ... but not now.
#models


from sklearn.preprocessing import StandardScaler, MinMaxScaler 
std_scal = StandardScaler()
mm_scal = MinMaxScaler()


##no tuning 
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score


from sklearn.model_selection import train_test_split



##regression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

#classifiers (non-tree)
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, SGDRegressor, SGDClassifier
from sklearn.svm import SVC


#tree-based classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import XGBRegressor

##regression models
lr = Ridge(alpha=0.001) 
rfr = RandomForestRegressor(max_depth=3, random_state=0)
xgbr = XGBRegressor()

##classifier models
lrc = RidgeClassifier()
gnb = GaussianNB()
lgr = LogisticRegression(random_state = 0)
svc = SVC()

#tree-based classifiers
rfc =  RandomForestClassifier(max_depth=3, random_state=0)
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()
xgbc = XGBClassifier()



In [4]:
y_cols = ['Unnamed: 0', 'game_id', 'mp_date', 'season', 'home_team', 'away_team',
       'home_odds', 'away_odds', 'home_goals', 'away_goals', 'goal_diff_target', 'home_win',
       'settled_in', ]

x_cols = ['CF%', 'CSh%', 'CSv%', 'FF%', 'FSh%', 'FSv%', 'GDiff',
       'GF%', 'PDO', 'PENDiff', 'SF%', 'SDiff', 'Sh%', 'Sv']
columns_to_scale = ['CF%', 'CSh%', 'CSv%', 'FF%', 'FSh%', 'FSv%', 'GDiff',
       'GF%', 'PDO', 'PENDiff', 'SF%', 'SDiff', 'Sh%', 'Sv']  ##same

In [5]:
X = pd.read_csv('/Users/joejohns/data_bootcamp/GitHub/final_project_nhl_prediction/Data/Shaped_Data/data_LJ.csv')

X['season'] = X['season'].apply(int)
X['game_id'] = X['game_id'].apply(int)
X['mp_date'] = X['mp_date'].apply(int)
X['goal_diff_target'] = X['home_goals'] - X['away_goals']

X.shape

(10385, 27)

In [32]:
def remove_early(X):
    filt_no_early  = (X['mp_date'].apply(lambda x : x% 10**4) < 900) | (1100 < X['mp_date'].apply(lambda x : x% 10**4))
    return X.loc[filt_no_early, : ].copy()  ## keep games < 900 and > 1100


In [7]:


x = np.array(X.loc[(X['season'] <= 20152016), x_cols].copy())
  #features test-train
Y = X.loc[(X['season'] <= 20152016), y_cols].copy()
   #targets
y = np.array(Y['home_win']).reshape(-1,1)


              
x_16 = X.loc[(X['season'] == 20162017), x_cols].copy()  #features test-train
Y_16 = X.loc[(X['season'] == 20162017), y_cols].copy()   #targets
y_16 = np.array(Y_16['home_win']).reshape(-1,1)

x_17 = X.loc[(X['season'] > 20162017), x_cols].copy()  #features test-train
Y_17 = X.loc[(X['season'] > 20162017), y_cols].copy()   #targets
y_17 = np.array(Y_17['home_win']).reshape(-1,1)




In [8]:
X.loc[(X['season'] >= 20162017), :]['season'].value_counts()

20162017    1060
20172018    1052
Name: season, dtype: int64

In [9]:
X.loc[(X['season'] <= 20152016), :]['season'].value_counts()

20142015    1046
20112012    1037
20152016    1036
20132014     994
20082009     971
20102011     962
20092010     946
Name: season, dtype: int64

In [10]:
X.loc[(X['season'] <= 20152016), :]['season'].value_counts().sum()

6992

In [11]:
print(X.shape)
print(x.shape, Y.shape, y.shape)
print(x_16.shape, y_16.shape)
print(x_17.shape, y_17.shape)
print('sum ', y.shape[0], y_16.shape[0], y_17.shape[0], ' is: ', y.shape[0] + y_16.shape[0]+y_17.shape[0])

(9104, 27)
(6992, 14) (6992, 13) (6992, 1)
(1060, 14) (1060, 1)
(1052, 14) (1052, 1)
sum  6992 1060 1052  is:  9104


In [12]:
Y['season'].value_counts()

20142015    1046
20112012    1037
20152016    1036
20132014     994
20082009     971
20102011     962
20092010     946
Name: season, dtype: int64

In [13]:
Y_16['season'].value_counts()

20162017    1060
Name: season, dtype: int64

In [14]:
Y_17['season'].value_counts()

20172018    1052
Name: season, dtype: int64

In [16]:

from sklearn.preprocessing import StandardScaler, MinMaxScaler 
std_scal = StandardScaler()
mm_scal = MinMaxScaler()


##no tuning 
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score


from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

#do standard/minmax scaling on X_train numeric columns ... better to do pipeline? 
x_train_sc = std_scal.fit_transform(x_train)
    
#fit the scaler from train portion to the test portion 
x_test_sc = std_scal.transform(x_test)








##classifier models
lrc = RidgeClassifier()
gnb = GaussianNB()
lgr = LogisticRegression(random_state = 0, max_iter = 1000)
svc = SVC(probability = True)

#tree-based classifiers
rfc =  RandomForestClassifier(max_depth=3, random_state=0)
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()
xgbc = XGBClassifier()



##quick checks 
#for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
def fit_model(model, print_results = True, tuned = False)    
    model.fit(x_train_sc, y_train.ravel())
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test_sc)
    y_predt= model.predict(x_train_sc)  
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test,y_pred)
    acct = accuracy_score(y_train, y_predt)
    f1t = f1_score(y_train,y_predt)
    if tuned:
        tuned= 'WITH tuning'
    else
        tuned = 'NO tuning'
    
    if print_results:  
        print(str(model)[0:12], tuned+ 'TEST acc, f1: ', acc, f1, 'training acct, f1t: ', acct, f1t)
  
    


SVC() no tuning TEST:  0.5589706933523946 0.6527855936972425 training :  0.5991417843733238 0.6873082287308229
RidgeClassif no tuning TEST:  0.557541100786276 0.6492917847025497 training :  0.5817986769175756 0.672821373618688
RandomForest no tuning TEST:  0.5589706933523946 0.6512153759185981 training :  0.5889504738065439 0.6790450928381963
GaussianNB() no tuning TEST:  0.5432451751250893 0.5725752508361204 training :  0.5716073663507957 0.6109126339720689
LogisticRegr no tuning TEST:  0.557541100786276 0.6480955088118249 training :  0.5794743429286608 0.6705882352941176
BaggingClass no tuning TEST:  0.5225160829163689 0.541838134430727 training :  0.9849812265331664 0.9862249918005904
GradientBoos no tuning TEST:  0.5568263045032166 0.6412037037037037 training :  0.6618988020740211 0.726457399103139
[11:40:15] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the de

/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifie no tuning TEST:  0.5325232308791994 0.6012195121951219 training :  0.9608439120328983 0.9649094696362762


In [30]:
x

array([[-1.07000000e+00,  3.88836773e+00, -1.50203584e+00, ...,
         0.00000000e+00,  6.54126895e+00, -2.47000000e+00],
       [-5.33000000e+00,  3.22158462e-01,  7.96868447e-01, ...,
        -8.69000000e+00,  7.75485350e-01,  2.28000000e+00],
       [-1.13900000e+01,  1.29385506e+00, -7.31934901e-01, ...,
        -1.05100000e+01,  2.16503340e+00, -2.47000000e+00],
       ...,
       [-1.07000000e+00, -1.70848125e-03, -6.54668512e-02, ...,
        -2.04000000e+00, -3.28207435e-01,  5.00000000e-01],
       [ 4.45000000e+00, -3.02955417e-01,  7.35640597e-01, ...,
         4.42000000e+00, -2.41020059e-01,  1.24000000e+00],
       [ 3.87000000e+00,  2.48248880e-01,  4.01685000e-01, ...,
         6.00000000e+00,  3.36489035e-01,  3.90000000e-01]])

In [17]:
#check on 2016

x_test2 = std_scal.transform(x_16).copy()
y_test2 = y_16.copy()


##quick checks 
for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
    #model.fit(x_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test2)
  
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
   
    #acct = accuracy_score(y_train, y_predt)
    #f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], '20162017, no tuning, TEST: ', acc, f1) #, 'training : ', acct, f1t)

SVC() 20162017, no tuning, TEST:  0.5943396226415094 0.684287812041116
RidgeClassif 20162017, no tuning, TEST:  0.5943396226415094 0.6838235294117646
RandomForest 20162017, no tuning, TEST:  0.5962264150943396 0.6862170087976539
GaussianNB() 20162017, no tuning, TEST:  0.5716981132075472 0.6092943201376936
LogisticRegr 20162017, no tuning, TEST:  0.5952830188679246 0.6847905951506247
BaggingClass 20162017, no tuning, TEST:  0.5452830188679245 0.5602189781021898
GradientBoos 20162017, no tuning, TEST:  0.5792452830188679 0.6579754601226995
XGBClassifie 20162017, no tuning, TEST:  0.5537735849056604 0.6119770303527481


In [18]:
#check on 2017

x_test2 = std_scal.transform(x_17).copy()
y_test2 = y_17.copy()


##quick checks 
for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
    #model.fit(x_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test2)
  
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
   
    #acct = accuracy_score(y_train, y_predt)
    #f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], '20172018, no tuning, TEST: ', acc, f1) #, 'training : ', acct, f1t)

SVC() 20172018, no tuning, TEST:  0.5874524714828897 0.6917613636363635
RidgeClassif 20172018, no tuning, TEST:  0.5855513307984791 0.6876790830945558
RandomForest 20172018, no tuning, TEST:  0.6036121673003803 0.7090020935101187
GaussianNB() 20172018, no tuning, TEST:  0.5731939163498099 0.628003314001657
LogisticRegr 20172018, no tuning, TEST:  0.5884030418250951 0.691814946619217
BaggingClass 20172018, no tuning, TEST:  0.5304182509505704 0.5711805555555556
GradientBoos 20172018, no tuning, TEST:  0.594106463878327 0.6876371616678859
XGBClassifie 20172018, no tuning, TEST:  0.5627376425855514 0.6423017107309487


#can do version where we get y regr version at beinning ... do all teh regression stuff ... 
then for later cells do y = v_win(y) for classifiers ... 

##regression models now ...

y = Y['goal_diff_target'].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#do standard/minmax scaling on X_train numeric columns ... better to do pipeline? 
x_train_sc = std_scal.fit_transform(x_train)
    
#fit the scaler from train portion to the test portion 
x_test_sc = std_scal.transform(x_test)


lr = Ridge(alpha=50000) 
rfr = RandomForestRegressor(max_depth=4, random_state=0)
xgbr = XGBRegressor()


##quick checks 
for model in [lr, rfr, xgbr]:
    model.fit(x_train_sc, y_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test_sc)
    print(y_pred[0:5])
    y_predw = v_win(y_pred)
    y_predt= model.predict(x_train_sc)  
    y_predwt = v_win(y_predt) 
    y_trainw = v_win(y_train)
    y_testw = v_win(y_test)  #same as usual win/loss
    acc = accuracy_score(y_testw, y_predw)
    f1 = f1_score(y_testw,y_predw)
    acct = accuracy_score(y_trainw, y_predwt)
    f1t = f1_score(y_trainw,y_predwt)
    
  
    print(str(model)[0:20], 'TEST: ', acc, f1 ,'training : ', acct, f1t)
    

In [19]:
## some tries at tuning here 


from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score


from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=12, stratify=y)

#do standard/minmax scaling on X_train numeric columns ... better to do pipeline? 
x_train_sc = std_scal.fit_transform(x_train)
    
#fit the scaler from train portion to the test portion 
x_test_sc = std_scal.transform(x_test)


##classifier models
lrc2 = RidgeClassifier(alpha =0.2)
gnb2 = GaussianNB()
lgr2 = LogisticRegression(random_state = 0, C =10**5, max_iter = 500)
svc2 = SVC(kernel = 'rbf', C =10, probability = True)  #why would you ever have it false? 

#tree-based classifiers
rfc2 =  RandomForestClassifier(max_depth=10, random_state=0, n_estimators = 40)
bc2 = BaggingClassifier(n_estimators  = 60, max_samples = 0.85)
gbc2 = GradientBoostingClassifier(learning_rate =0.1, n_estimators = 40, max_depth =4 )
xgbc2 = XGBClassifier(n_estimators= 45, eta=0.05)

##quick checks 
for model in [svc2, lrc2, rfc2, gnb2, lgr2, bc2, gbc2, xgbc2]:
    model.fit(x_train_sc, y_train.ravel())
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test_sc)
    y_predt= model.predict(x_train_sc)  
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test,y_pred)
    acct = accuracy_score(y_train, y_predt)
    f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], 'with tuning TEST: ', acc, f1, 'training : ', acct, f1t)

SVC(C=10, pr with tuning TEST:  0.5668334524660472 0.6666666666666667 training :  0.6264974074736277 0.7127732710023373
RidgeClassif with tuning TEST:  0.5675482487491065 0.6606842400448683 training :  0.5758984444841766 0.6692693809258226
RandomForest with tuning TEST:  0.5668334524660472 0.6521239954075775 training :  0.8119077418201323 0.8471818710052295
GaussianNB() with tuning TEST:  0.5689778413152251 0.6045901639344262 training :  0.5649919542284999 0.606119475473531
LogisticRegr with tuning TEST:  0.5661186561829878 0.6564799094510471 training :  0.573574110495262 0.6666666666666667


/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


BaggingClass with tuning TEST:  0.5318084345961401 0.5888261142498431 training :  0.9998212050777758 0.9998372130880677
GradientBoos with tuning TEST:  0.5725518227305219 0.6617647058823529 training :  0.6676202395851958 0.7381321312860967
[11:40:34] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifie with tuning TEST:  0.5539671193709793 0.6450511945392492 training :  0.7201859467191132 0.7769066286528866


In [20]:
#check on 2016

x_test2 = std_scal.transform(x_16).copy()
y_test2 = y_16.copy()


##quick checks 
#for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
for model in [svc2, lrc2, rfc2, gnb2, lgr2, bc2, gbc2, xgbc2]:
    #model.fit(x_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test2)
  
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
   
    #acct = accuracy_score(y_train, y_predt)
    #f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], '20162017, tuned, TEST: ', acc, f1) #, 'training : ', acct, f1t)

SVC(C=10, pr 20162017, tuned, TEST:  0.5764150943396227 0.6715435259692759
RidgeClassif 20162017, tuned, TEST:  0.5924528301886792 0.6855895196506551
RandomForest 20162017, tuned, TEST:  0.5754716981132075 0.6631736526946107
GaussianNB() 20162017, tuned, TEST:  0.5707547169811321 0.6094420600858369
LogisticRegr 20162017, tuned, TEST:  0.5877358490566038 0.6807888970051132
BaggingClass 20162017, tuned, TEST:  0.5490566037735849 0.6062602965403624
GradientBoos 20162017, tuned, TEST:  0.5650943396226416 0.6567386448250185
XGBClassifie 20162017, tuned, TEST:  0.5660377358490566 0.6541353383458647


In [21]:
#check on 2017

x_test2 = std_scal.transform(x_17).copy()
y_test2 = y_17.copy()


##quick checks 
#for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
for model in [svc2, lrc2, rfc2, gnb2, lgr2, bc2, gbc2, xgbc2]:
    #model.fit(x_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test2)
  
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
   
    #acct = accuracy_score(y_train, y_predt)
    #f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12],  '20172018, tuned, TEST: ', acc, f1) #, 'training : ', acct, f1t)

SVC(C=10, pr 20172018, tuned, TEST:  0.5836501901140685 0.6857962697274033
RidgeClassif 20172018, tuned, TEST:  0.5960076045627376 0.6975088967971531
RandomForest 20172018, tuned, TEST:  0.5931558935361216 0.6857562408223201
GaussianNB() 20172018, tuned, TEST:  0.5674904942965779 0.6211490424646129
LogisticRegr 20172018, tuned, TEST:  0.5960076045627376 0.6992215145081387
BaggingClass 20172018, tuned, TEST:  0.5484790874524715 0.6141348497156783
GradientBoos 20172018, tuned, TEST:  0.5950570342205324 0.6921965317919075
XGBClassifie 20172018, tuned, TEST:  0.5988593155893536 0.6901615271659325


In [22]:
#Let's go with lgr, lgr2 (tuned or not) for betting investigation

In [23]:
x_16 = std_scal.transform(x_16).copy()


In [26]:
svc2.predict_proba(x_16)

array([[0.52407527, 0.47592473],
       [0.58086487, 0.41913513],
       [0.41006043, 0.58993957],
       ...,
       [0.37652706, 0.62347294],
       [0.44296909, 0.55703091],
       [0.3999811 , 0.6000189 ]])

In [28]:
gnb.predict_proba(x_16)[0:, 0]

array([0.00380656, 0.32217664, 0.21205196, ..., 0.07010458, 0.54824593,
       0.05309209])

In [29]:
y_16

array([[1.],
       [1.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]])

In [ ]:
##make betting strategy on x_16 test it on x_17  ... and other seasons if needed

In [ ]:
def make_dic(model, year):
    
    
    predict_proba = [round(x,4) for x in model.predict_proba(x_16)[0:, 0]]
    
    dic['lgr_pred'] = list(model.predict(x_16))
    dic['actual'] = [ x[0] for x in list(y_16)]
    dic['lgr_conf_0'] =[round(x,4) for x in model.predict_proba(x_16)[0:, 0]]
    dic['lgr_conf_1'] = [round(x,4) for x in model.predict_proba(x_16)[0:, 1]]
    dic['away_odds'] = list(Y_16['away_odds'])
    dic['home_odds'] = list(Y_16['home_odds'])
    df_svc = pd.DataFrame(dic_svc)

In [ ]:


df_svc['away_impl_proba'] = v_impl_proba(df_svc['away_odds'])
df_svc['home_impl_proba'] = v_impl_proba(df_svc['home_odds'])


df_svc['home_model-odds'] = df_svc['lgr_conf_1'] - df_svc['home_impl_proba']
df_svc['away_model-odds'] = df_svc['lgr_conf_0'] - df_svc['away_impl_proba']

In [ ]:
df_svc

In [ ]:
def implied_proba(odds):
    if odds > 0: 
        return 100/(odds+100)    #bet 100 to get 100+odds; profit = odds
    if odds < 0:
        return (-odds)/(-odds + 100)   #bet |odds| to get 100+|odds|; profit = 100
    
v_impl_proba = np.vectorize(implied_proba)

In [ ]:


dic['lgr_pred'] = list(lgr.predict(x_16))
dic['gnb_pred'] = list(gnb.predict(x_16))
dic['svc_proba'] = list(svc.predict_proba(x_16)[:,1])


In [ ]:
predictions = gnb.predict(x_17)
actual = y_17
confusionMatrix = confusion_matrix(actual, predictions)
display(confusionMatrix)

tn = confusionMatrix[0][0]
fp = confusionMatrix[0][1]
fn = confusionMatrix[1][0]
tp = confusionMatrix[1][1]
actualYes = fn + tp 
actualNo = tn + fp
predictedYes = fp + tp
predictedNo = tn + fn

print('When home team wins, classifier predicts they will win %6.2f%% of the time' % (tp / actualYes * 100))
print('When home team loses, classifier predicts they will win %6.2f%% of the time' % (fp / actualNo * 100))
print('When home team loses, classifier predicts they will lose %6.2f%% of the time' % (tn / actualNo * 100))
print('When classifer predicts home team will win, home team actually wins %6.2f%% of the time' % (tp / predictedYes * 100))
print('When classifer predicts home team will lose, home team actually loses %6.2f%% of the time' % (tn / predictedNo * 100))